In [1]:
!git clone https://github.com/imostafizur/CSE498R.git

^C


## Pneumonia Detection with InceptionV3

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import TerminateOnNaN
from tensorflow.keras.callbacks import ProgbarLogger
from tensorflow.keras.callbacks import History
from tensorflow.keras.callbacks import BaseLogger
from tensorflow.keras.callbacks import RemoteMonitor
from tensorflow.keras.callbacks import CallbackList
from tensorflow.keras.callbacks import Callback

In [1]:
image = ('Pneumonia')

In [2]:
print('Total number of images in Pneumonia folder:', len(os.listdir('Pneumonia')))
print('Total number of images in Normal folder:',
      len(os.listdir('Pneumonia/NORMAL')))
print('Total number of images in Pneumonia folder:',
      len(os.listdir('Pneumonia/PNEUMONIA')))


Total number of images in Pneumonia folder: 2
Total number of images in Normal folder: 1342
Total number of images in Pneumonia folder: 3876


In [7]:
# InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

# Compile the model
model.compile(loss='binary_crossentropy',
                optimizer=Adam(lr=0.0001),
                metrics=['accuracy'])

<keras.engine.input_layer.InputLayer object at 0x0000021C891033D0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000021C89101ED0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000021C891012D0> False
<keras.layers.core.activation.Activation object at 0x0000021CF189C280> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000021C89101750> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000021C89103F10> False
<keras.layers.core.activation.Activation object at 0x0000021C88A2F730> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000021CF180AEC0> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000021C88A2F820> False
<keras.layers.core.activation.Activation object at 0x0000021CEE7CB160> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000021C89102950> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x0000021C88A2E9

In [8]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [9]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [10]:
# Data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
# Data augmentation
val_datagen = ImageDataGenerator(rescale=1./255)


In [13]:
# Data augmentation
train_generator = train_datagen.flow_from_directory(
        'Pneumonia',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.


In [14]:
# Data augmentation
test_generator = test_datagen.flow_from_directory(
        'Pneumonia',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.


In [15]:
# Data augmentation
val_generator = val_datagen.flow_from_directory(
        'Pneumonia',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.


In [16]:
# Callbacks
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                mode='auto')

In [17]:
# Callbacks
earlystop = EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=0,
                            mode='auto')

In [18]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                factor=0.2,
                                patience=3,
                                verbose=0,
                                mode='auto')

In [19]:
# Callbacks
tensorboard = TensorBoard(log_dir='logs',
                            histogram_freq=0,
                            write_graph=True,
                            write_images=True)

In [20]:
# Callbacks
csv_logger = CSVLogger('log.csv',
                        append=False,
                        separator=',')

In [23]:

# Callbacks
lambda_callback = LambdaCallback(on_epoch_begin=None,
                                    on_epoch_end=None,
                                    on_batch_begin=None,
                                    on_batch_end=None,
                                    on_train_begin=None,
                                    on_train_end=None)

In [24]:
# Callbacks
terminate_on_nan = TerminateOnNaN()

In [25]:
# Training
history = model.fit_generator(
        train_generator,
        steps_per_epoch=5216 // 32,
        epochs=10,
        validation_data=val_generator,
        validation_steps=624 // 32,
        callbacks=[checkpoint, earlystop, reduce_lr, tensorboard, csv_logger, lambda_callback, terminate_on_nan])

C:\Users\Mostafizur\AppData\Local\Temp\ipykernel_17800\4284759501.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
 33/163 [=====>........................] - ETA: 4:46 - loss: 1.3373 - accuracy: 0.7538

In [ ]:
# Save the model h5
model.save('Pneumonia_InceptionV3.h5')